In [2]:
import pandas as pd
import scipy.stats as stats
import statsmodels.stats.multitest as sm_multi
import scipy.stats as stats 
import math

In [4]:
# Read data

supp_12_17_excel = '../Priya_et_al_Supplementary_Tables/Supplementary Tables S12-S17.xlsx'
supp_1_excel = '../Priya_et_al_Supplementary_Tables/Supplementary Table S1.xlsx'

crc_genes=pd.read_excel(supp_12_17_excel,engine='openpyxl',sheet_name='S12',index_col=0)
ibd_genes=pd.read_excel(supp_12_17_excel,engine='openpyxl',sheet_name='S14',index_col=0)
ibs_genes=pd.read_excel(supp_12_17_excel,engine='openpyxl',sheet_name='S16',index_col=0)

crc_metadata = pd.read_excel(supp_1_excel,engine='openpyxl',sheet_name='CRC_metadata',index_col=0)
ibd_metadata = pd.read_excel(supp_1_excel,engine='openpyxl',sheet_name='IBD_metadata',index_col=0)
ibs_metadata = pd.read_excel(supp_1_excel,engine='openpyxl',sheet_name='IBS_metadata',index_col=1)

In [5]:
crc_genes.head()

,B01,B02,B03,B04,B05,B06,B07,B08,B09,B10,...,s74,s85,s86,s87,s88,s89,s90,s94,s95,s96
TSPAN6,235,815,351,629,91,175,1240,600,1325,672,...,1426,1230,206,809,103,1028,14,1985,369,291
TNMD,23,15,188,12,4,10,24,11,13,8,...,34,14,7,0,6,46,0,16,3,7
DPM1,184,166,146,124,150,121,536,212,237,275,...,299,345,283,325,223,327,29,606,344,198
SCYL3,263,389,185,231,480,340,432,335,559,427,...,546,267,298,529,250,416,91,428,493,287
C1orf112,98,73,69,70,457,106,233,109,114,148,...,289,198,133,323,103,178,10,247,289,115


In [6]:
ALPHA = 0.05
EQUAL_VARIANCE = False

# Do t-test to find out interesting human genes that are different in 'controls' and 'cases'

def getInterestingGenes(data, metadata, classificationColumn, classificationValue, alpha=ALPHA):
    patient_ids_1 = metadata.query(f'{classificationColumn} == "{classificationValue}"').index.to_list()
    patient_ids_2 = metadata.query(f'{classificationColumn} != "{classificationValue}"').index.to_list()
    data1 = data.filter(items=[*patient_ids_1])
    data2 = data.filter(items=[*patient_ids_2])
    columns = ['statistic', 'p', 'gene']
    summary = pd.DataFrame(columns=columns)

    for i in range(len(data2)):
        result = stats.ttest_ind(data2.iloc[i].to_list(), data1.iloc[i].to_list(), equal_var=EQUAL_VARIANCE)
        summary.loc[i] = [result.statistic, result.pvalue, data2.index[i]]
  
    return summary

In [19]:
crc_interesting_genes = getInterestingGenes(crc_genes, crc_metadata, 'Description', 'normal')
ibd_interesting_genes = getInterestingGenes(ibd_genes, ibd_metadata, 'Diagnosis', 'nonIBD')
ibs_interesting_genes = getInterestingGenes(ibs_genes, ibs_metadata, 'Cohort', 'Healthy')

In [20]:
crc_interesting_genes.head()

,statistic,p,gene
0,5.511634,9.322244e-07,TSPAN6
1,0.403856,6.873585e-01,TNMD
2,6.119861,6.768268e-08,DPM1
3,2.292109,2.436865e-02,SCYL3
4,5.874597,3.423666e-07,C1orf112


In [22]:
# Perform Bonferonni correction to narrow down the list

bonferroni_alpha_crc = 0.05/len(crc_interesting_genes)

crc_interesting_genes_bonferroni_correct = \
crc_interesting_genes[crc_interesting_genes['p'] < bonferroni_alpha_crc]

crc_interesting_genes_bonferroni_correct.head()

,statistic,p,gene
0,5.511634,9.322244e-07,TSPAN6
2,6.119861,6.768268e-08,DPM1
4,5.874597,3.423666e-07,C1orf112
7,5.713198,3.482805e-07,FUCA2
9,6.172941,5.229376e-08,NFYA


In [23]:
crc_interesting_genes = crc_interesting_genes.dropna()
ibd_interesting_genes = ibd_interesting_genes.dropna()
ibs_interesting_genes = ibs_interesting_genes.dropna()

In [26]:
# Peform FDR correction to narrow down further

[Res, P_cor] = sm_multi.fdrcorrection(crc_interesting_genes['p'])
crc_interesting_genes['corrected p']  = P_cor
crc_interesting_genes_fdr_corrected = \
crc_interesting_genes[crc_interesting_genes['corrected p'] < bonferroni_alpha_crc]

crc_interesting_genes_fdr_corrected.head()

,statistic,p,gene,corrected p
2,6.119861,6.768268e-08,DPM1,6.412883e-07
4,5.874597,3.423666e-07,C1orf112,2.499767e-06
7,5.713198,3.482805e-07,FUCA2,2.533477e-06
9,6.172941,5.229376e-08,NFYA,5.159560e-07
23,6.155734,1.148949e-07,HS3ST1,9.948982e-07


In [27]:
# Final list after correction

len(crc_interesting_genes_fdr_corrected)

2610